In [1]:
# Initialize DSPy
import sys
import dspy
sys.path.append('..')  # Add parent directory to path
from dspy_utils import load_dspy_config
load_dspy_config("../dspy_config.json")

In [3]:
# Load analysis plan template
with open('../templates/analysis_plan_template.md', 'r') as f:
    template_content = f.read()

# DSPy signature for plan generation
class GenerateAnalysisPlan(dspy.Signature):
    """Generate a population PK/PD analysis plan based on study parameters"""
    drug_name: str = dspy.InputField(desc="Name of the drug")
    indication: str = dspy.InputField(desc="Disease indication")
    competitor: str = dspy.InputField(desc="Competitor drug name")
    trial_type: str = dspy.InputField(desc="Type of clinical trial")
    template: str = dspy.InputField(desc="Analysis plan template")
    analysis_plan: str = dspy.OutputField(desc="Generated analysis plan")

In [ ]:
# Generate example plans
generate_plan = dspy.ChainOfThought(GenerateAnalysisPlan)

# Test with different parameters using made-up drug names
examples = [
    {"indication": "Type 2 Diabetes", "competitor": "Metformin", "trial_type": "Phase 2 dose-finding", "drug_name": "Glucamax"},
    # {"indication": "Hypertension", "competitor": "Lisinopril", "trial_type": "Phase 3 efficacy", "drug_name": "Cardiogen"},
    # {"indication": "Oncology", "competitor": "Pembrolizumab", "trial_type": "Phase 1 dose-escalation", "drug_name": "Oncotarget"}
]

# Create output directory
import os
output_dir = "generated_data"
os.makedirs(output_dir, exist_ok=True)

In [5]:
# DSPy signature for R script generation
class GenerateTrialSimulation(dspy.Signature):
    """Generate R script for clinical trial simulation using mrgsolve"""
    analysis_plan: str = dspy.InputField(desc="Analysis plan text")
    indication: str = dspy.InputField(desc="Disease indication")
    trial_type: str = dspy.InputField(desc="Type of clinical trial")
    r_script: str = dspy.OutputField(desc="Complete R script using mrgsolve for trial simulation. Output of script should be standard SAS SDTM datsets for PK PD analysis.")

In [ ]:
# Generate and save analysis plans and R scripts
generate_simulation = dspy.ChainOfThought(GenerateTrialSimulation)

for i, params in enumerate(examples):
    # Create drug-specific subdirectory
    drug_dir = f"{output_dir}/{params['drug_name']}"
    os.makedirs(drug_dir, exist_ok=True)
    
    # Generate analysis plan
    plan_result = generate_plan(
        drug_name=params["drug_name"],
        indication=params["indication"],
        competitor=params["competitor"], 
        trial_type=params["trial_type"],
        template=template_content
    )
    
    # Generate R script
    script_result = generate_simulation(
        analysis_plan=plan_result.analysis_plan,
        indication=params["indication"],
        trial_type=params["trial_type"]
    )
    
    # Save analysis plan in drug subdirectory
    plan_filename = f"{drug_dir}/analysis_plan.md"
    with open(plan_filename, 'w') as f:
        f.write(plan_result.analysis_plan)
    
    # Clean and save R script in drug subdirectory (remove markdown code blocks)
    script_filename = f"{drug_dir}/simulation.R"
    clean_script = script_result.r_script.replace('```r', '').replace('```R', '').replace('```', '').strip()
    with open(script_filename, 'w') as f:
        f.write(clean_script)
    
    print(f"Generated: {params['drug_name']} ({params['indication']}) - files saved to {drug_dir}/")

print(f"\nAll files organized in subdirectories under: {output_dir}/")